
# **<center>G-Research Crypto Forecasting</center>**

<a id="TOC"></a> <br>
# <center>Table of Contents</center>
<h4>Asset Details</h4>

* [Importing, Reading, and Verifying the Data](#0)
* [Universal Downcasting Function for Dataframes](#1)
  * [Asset Data Downcasting](#2)
* [Universal Basic Summary Function for Dataframes](#3)
  * [Asset Data Basic Summary](#4)
* [Calculate Percentage Weight for Each Coin.](#5)
* [Plotting the Amount of Weight Each Crypto, Received in the Metric.](#6)

<h4>Train Data</h4>

* [Reading and Downcasting Train Data](#7)
    * [Converting Timestamp to G - Research Location](#8)
    * [Segregation of Timestamp into Date, Year, Month, Day & Hour](#9) 
* [Basic Summary of Train Data](#10)
* [Setting Date As Index for ARIMA, LSTM and Visualization also removing NaN values](#11)
* [Histroy of CRYPTO Data from 2018-2021(Sep)](#12)
* [Opening Prize of All CRYPTO from 2018 - 2021(Sep)](#13)
* [Closing Prize of All CRYPTO from 2018 - 2021(Sep)](#14)
* [Opening Prize of All CRYPTO in Past 90 Days (24 Jun - 21 Sep) Hourly](#15)
* [Closing Prize of All CRYPTO in Past 90 Days (24 Jun - 21 Sep) Hourly](#16)
* [Getting Only Necessary Data for ARIMA Model](#17)
* [Past 3 Months Bitcoin Data with High Close & Low Close](#18)
    * [24 June to 26 June Bitcoin Data](#19)
    * [27 June to 30 June Bitcoin Data](#20)
    * [1 July to 5 July Bitcoin Data](#21)
    * [6 July to 10 June Bitcoin Data](#22)
    * [11 July to 15 July Bitcoin Data](#23)
    * [16 July to 20 July Bitcoin Data](#24)
    * [21 July to 25 July Bitcoin Data](#25)
    * [26 July to 28 July Bitcoin Data](#26)
    * [29 July to 31 July Bitcoin Data](#27)
    * [1 August to 5 August Bitcoin Data](#28)
    * [6 August to 10 August Bitcoin Data](#29)
    * [11 August to 15 August Bitcoin Data](#30)
    * [16 August to 20 August Bitcoin Data](#31)
    * [21 August to 25 August Bitcoin Data](#32)
    * [26 August to 28 August Bitcoin Data](#33)
    * [29 August to 31 August Bitcoin Data](#34)
    * [1 September to 5 September Bitcoin Data](#35)
    * [6 September to 10 September Bitcoin Data](#36)
    * [11 September to 15 September Bitcoin Data](#37)
    * [16 September to 18 September Bitcoin Data](#38)
    * [19 September to 21 September Bitcoin Data](#39)
* [Final Data For ARIMA Model](#40)
* [Identifying P value with ADF](#41)
* [1st Order Differencing & 2nd Order Differencing Autocorrelation D and Q value](#42)
* [1st Order Partial Differencing Partial Autocorrelation](#43)
* [1st Order Partial Differencing Autocorrelation](#44)
* [ARIMA Model](#45)
* [Residuals Plotting for ARIMA](#46)
* [RMSE](#47)
* [Forecasting Bitcoin Data](#48)

<h4>To Be Continued.......</h4>

<a id="AssetDetails"></a> <br>
# <center>Asset Details</center>

<a id="0"></a> <br>
# Importing, Reading, and Verifying the Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from tzlocal import get_localzone
from statsmodels.tsa.stattools import adfuller
from numpy import log
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import statsmodels.api as sm
import statsmodels.tsa.arima.model as stats
from statsmodels.tsa.arima.model import ARIMA
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from math import sqrt

assetDetailsData = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')

<a id="1"></a> <br>
# Universal Downcasting Function for Dataframes

In [ ]:
def downcastMemoryUsage(dataFrame):
    startMemoryOptimization = dataFrame.memory_usage().sum() / 1024 ** 2
    print('Memory usage of dataframe is: {:.2f} MB'.format(startMemoryOptimization))
    subTypeInt = ['uint8','uint16','uint32','uint64','int8','int16','int32','int64']
    subTypeFloat = ['float16','float32','float64']
    for column in dataFrame.columns:
        columnType = str(dataFrame[column].dtypes)
        maximumColumn = dataFrame[column].max()
        minimumColumn = dataFrame[column].min()
        if 'int' in columnType:
            for element in subTypeInt:
                if minimumColumn > np.iinfo(element).min and maximumColumn < np.iinfo(element).max:
                    dataFrame[column] = dataFrame[column].astype(element)
                    break
        elif 'float' in columnType:
            for element in subTypeFloat:
                if minimumColumn > np.finfo(element).min and maximumColumn < np.finfo(element).max:
                    dataFrame[column] = dataFrame[column].astype(element)
                    break
        elif 'object' in columnType:
            if column =='date':
                dataFrame['date'] = pd.to_datetime(dataFrame['date'],format='%Y-%m-%d')
            else:
                numberOfUnique = len(dataFrame[column].unique())
                numberOfTotal = len(dataFrame[column])
                if numberOfUnique / numberOfTotal < 0.5:
                    dataFrame[column] = dataFrame[column].astype('category')
    endMemoryOptimization = dataFrame.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(endMemoryOptimization))
    print('Compressed by: {:.2f} %'.format(100*(startMemoryOptimization - endMemoryOptimization) / startMemoryOptimization))
    
    return dataFrame

**<p id="2"><li>Asset Data Downcasting</li></p>**

In [ ]:
downcastMemoryUsage(assetDetailsData)

<a id="3"></a> <br>
# Universal Basic Summary Function for Dataframes

In [ ]:
def basicSummary(dataFrameForSummary):
    print(f'Shape : {dataFrameForSummary.shape}')
    summary = pd.DataFrame(dataFrameForSummary.dtypes, columns=['Data Type'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index': 'Feature'})
    summary['Num of Nulls'] = dataFrameForSummary.isnull().sum().values
    summary['Num of Unique'] = dataFrameForSummary.nunique().values
    summary['First Value'] = dataFrameForSummary.loc[0].values
    summary['Second Value'] = dataFrameForSummary.loc[1].values
    summary['Third Value'] = dataFrameForSummary.loc[2].values
    summary['Fourth Value'] = dataFrameForSummary.loc[3].values
    summary['Fifth Value'] = dataFrameForSummary.loc[4].values
    return summary

**<p id="4"><li>Asset Data Basic Summary</li></p>**

In [ ]:
basicSummary(assetDetailsData)

<a id="5"></a> <br>
# Calculate Percentage Weight for Each Coin

In [ ]:
assetDetailsData.sort_values(by=['Weight'],ascending=False,inplace=True)
# df[percent] = (df['column_name'] / df['column_name'].sum()) * 100
assetDetailsData['coinWeightPercent'] = (assetDetailsData['Weight'] / assetDetailsData['Weight'].sum()) * 100
assetDetailsData

<a id="6"></a> <br>
# Plotting the Amount of Weight Each Crypto, Received in the Metric

In [ ]:
fig = plt.figure()
ax = plt.gca()
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 5, box.height * 3])
colors = sns.color_palette('colorblind')[0:14]
labels = ['Bitcoin', 'Ethereum', 'Cardano', 'Binance Coin', 'Dogecoin', 'Bitcoin Cash', 'Litecoin', 'Ethereum Classic',
          'Stellar', 'TRON', 'Monero', 'EOS.IO', 'IOTA', 'Maker']
explode = (0.3, 0.3, 0.2, 0.2, 0.1, 0.1, 0.1, 0.0, 0.1, 0.0, 0.1, 0.0, 0.1, 0.0)
plt.pie(assetDetailsData['coinWeightPercent'], colors=colors, autopct='%.0f%%', labels=labels, explode=explode,
        startangle=30, shadow=True, textprops={'fontweight': 'semibold', 'fontsize': 15},
        wedgeprops={'linewidth': 2, 'edgecolor': 'k'}, labeldistance=1.1)
plt.title("Amount of Weight Each Crypto, Received in the Metric.", fontweight="bold", fontsize=22, pad=21)
plt.axis('equal')
plt.show()

<a id="TrainData"></a> <br>
# <center>Train Data</center>

<a id="7"></a> <br>
# Downcasting Train Data

**<p id="8"><li>Converting Timestamp to G - Research Location</li></p>**
**<p id="9"><li>Segregation of Timestamp into Date, Year, Month, Day & Hour</li></p>**

In [ ]:
trainData = pd.read_csv('../input/g-research-crypto-forecasting/train.csv')
# trainData['timestamp']=pd.to_datetime(trainData['timestamp'], unit='s', yearfirst=True, errors='coerce').astype('datetime64[ns]')
trainData['DateAndTime'] = pd.to_datetime(trainData['timestamp'], unit='s',utc = True).apply(lambda x:x.tz_convert('Europe/London'))
trainData['Date'] = trainData.DateAndTime.dt.date
trainData['Year'] = trainData.DateAndTime.dt.year
trainData['Month'] = trainData.DateAndTime.dt.month
trainData['Day'] = trainData.DateAndTime.dt.day
trainData['Hour'] = trainData.DateAndTime.dt.hour
# trainData['Minute'] = trainData.DateAndTime.dt.minute
# trainData['Date'] = pd.to_datetime(trainData['timestamp'],unit='s', utc = True).apply(lambda x: x.date())
downcastMemoryUsage(trainData)
trainData.iloc[::5000000, :]

<a id="10"></a> <br>
# Basic Summary of Train Data

In [ ]:
basicSummary(trainData)

<a id="11"></a> <br>
# Setting Date As Index for ARIMA, LSTM and Visualization also removing NaN values

In [ ]:
trainData.set_index(['DateAndTime'], inplace = True)
trainData['Target'].interpolate(method='time', inplace=True)
trainData['VWAP'].interpolate(method='time', inplace=True)

<a id="12"></a> <br>
# Histroy of CRYPTO Data from 2018-2021(Sep)

In [ ]:
assetsNamesDictionary = {row["Asset_Name"]:row["Asset_ID"] for x, row in assetDetailsData.iterrows()}

assetNames = ['Bitcoin', 'Ethereum', 'Cardano', 'Binance Coin', 'Dogecoin', 'Bitcoin Cash', 'Litecoin', 'Ethereum Classic',
          'Stellar', 'TRON', 'Monero', 'EOS.IO', 'IOTA', 'Maker']

fromToListTimes = []
for crypto in assetNames:
    cryptoDataFrame = trainData[trainData["Asset_ID"] == assetsNamesDictionary[crypto]]
    fromTime = cryptoDataFrame.index[0]
    toTime = cryptoDataFrame.index[-1]
    fromToListTimes.append([crypto, fromTime, toTime])
fromToDataframe = pd.DataFrame(fromToListTimes)
fromToDataframe.columns = ["AssetName", "StartsFrom", "EndsTo"]
fromToDataframe

<a id="13"></a> <br>
# Opening Prize of All CRYPTO from 2018 - 2021(Sep)

In [ ]:
f = plt.figure(figsize=(20,45))
for x, crypto in enumerate(assetNames):
    cryptoDataFrame = trainData[trainData["Asset_ID"]==assetsNamesDictionary[crypto]]
    ax = f.add_subplot(7,2,x+1)
    sns.lineplot(x = 'Date', y = 'Open', hue = 'Year', data = cryptoDataFrame,ci=None)
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel('Value in USD')
    plt.title(crypto)
    plt.grid(b = True, axis = 'y')

plt.tight_layout()
plt.show()

<a id="14"></a> <br>
# Closing Prize of All CRYPTO from 2018 - 2021(Sep)

In [ ]:
f = plt.figure(figsize=(20,45))
for x, crypto in enumerate(assetNames):
    cryptoDataFrame = trainData[trainData["Asset_ID"]==assetsNamesDictionary[crypto]]
    ax = f.add_subplot(7,2,x+1)
    sns.lineplot(x = 'Date', y = 'Close', hue = 'Year', data = cryptoDataFrame,ci=None)
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel('Value in USD')
    plt.title(crypto)
    plt.grid(b = True, axis = 'y')

plt.tight_layout()
plt.show()

<a id="15"></a> <br>
# Opening Prize of All CRYPTO in Past 90 Days (24 Jun - 21 Sep) Hourly

In [ ]:
f = plt.figure(figsize=(20,45))
for x, crypto in enumerate(assetNames):
    cryptoDataFrame = trainData[trainData["Asset_ID"]==assetsNamesDictionary[crypto]]
    cryptoDataFrame = cryptoDataFrame.loc['2021-06-24 00:01:00+00:00': '2021-09-21 00:01:00+00:00']
    ax = f.add_subplot(7,2,x+1)
    sns.lineplot(x = 'Date', y = 'Open', hue = 'Hour', data = cryptoDataFrame,ci=None)
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel('Value in USD')
    plt.title(crypto)
    plt.grid(b = True, axis = 'y')

plt.tight_layout()
plt.show()

<a id="16"></a> <br>
# Closing Prize of All CRYPTO in Past 90 Days (24 Jun - 21 Sep) Hourly

In [ ]:
f = plt.figure(figsize=(20,45))
for x, crypto in enumerate(assetNames):
    cryptoDataFrame = trainData[trainData["Asset_ID"]==assetsNamesDictionary[crypto]]
    cryptoDataFrame = cryptoDataFrame.loc['2021-06-24 00:01:00+00:00': '2021-09-21 00:01:00+00:00']
    ax = f.add_subplot(7,2,x+1)
    sns.lineplot(x = 'Date', y = 'Close', hue = 'Hour', data = cryptoDataFrame,ci=None)
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel('Value in USD')
    plt.title(crypto)
    plt.grid(b = True, axis = 'y')

plt.tight_layout()
plt.show()

<a id="17"></a> <br>
# Getting only necessary data for ARIMA Model

In [ ]:
trainData.reset_index(drop=True, inplace=True)
trainData['Date'] = trainData.Date.astype(str)

newTrainData = trainData[['Asset_ID','Date','Open','Close','High','Low','Month']]

<a id="18"></a> <br>
# Past 3 month Bitcoin Data with High Close and Low Close

**<p id="19"><li>24 June to 26 June Bitcoin Data</li></p>**


In [ ]:
trainDataFor24Jun = newTrainData[(newTrainData.Date == '2021-06-24') & (newTrainData.Asset_ID == 1)]
trainDataFor24JunHigh = trainDataFor24Jun.loc[trainDataFor24Jun[['High']].idxmax()]
trainDataFor24JunLow = trainDataFor24Jun.loc[trainDataFor24Jun[['Low']].idxmin()]

trainDataFor25Jun = newTrainData[(newTrainData.Date == '2021-06-25') & (newTrainData.Asset_ID == 1)]
trainDataFor25JunHigh = trainDataFor25Jun.loc[trainDataFor25Jun[['High']].idxmax()]
trainDataFor25JunLow = trainDataFor25Jun.loc[trainDataFor25Jun[['Low']].idxmin()]

trainDataFor26Jun = newTrainData[(newTrainData.Date == '2021-06-26') & (newTrainData.Asset_ID == 1)]
trainDataFor26JunHigh = trainDataFor26Jun.loc[trainDataFor26Jun[['High']].idxmax()]
trainDataFor26JunLow = trainDataFor26Jun.loc[trainDataFor26Jun[['Low']].idxmin()]

**<p id="20"><li>27 June to 30 June Bitcoin Data</li></p>**

In [ ]:
trainDataFor27Jun = newTrainData[(newTrainData.Date == '2021-06-27') & (newTrainData.Asset_ID == 1)]
trainDataFor27JunHigh = trainDataFor27Jun.loc[trainDataFor27Jun[['High']].idxmax()]
trainDataFor27JunLow = trainDataFor27Jun.loc[trainDataFor27Jun[['Low']].idxmin()]

trainDataFor28Jun = newTrainData[(newTrainData.Date == '2021-06-28') & (newTrainData.Asset_ID == 1)]
trainDataFor28JunHigh = trainDataFor28Jun.loc[trainDataFor28Jun[['High']].idxmax()]
trainDataFor28JunLow = trainDataFor28Jun.loc[trainDataFor28Jun[['Low']].idxmin()]

trainDataFor29Jun = newTrainData[(newTrainData.Date == '2021-06-29') & (newTrainData.Asset_ID == 1)]
trainDataFor29JunHigh = trainDataFor29Jun.loc[trainDataFor29Jun[['High']].idxmax()]
trainDataFor29JunLow = trainDataFor29Jun.loc[trainDataFor29Jun[['Low']].idxmin()]

trainDataFor30Jun = newTrainData[(newTrainData.Date == '2021-06-30') & (newTrainData.Asset_ID == 1)]
trainDataFor30JunHigh = trainDataFor30Jun.loc[trainDataFor30Jun[['High']].idxmax()]
trainDataFor30JunLow = trainDataFor30Jun.loc[trainDataFor30Jun[['Low']].idxmin()]

**<p id="21"><li>1 July to 5 July Bitcoin Data</li></p>**

In [ ]:
trainDataFor1Jul = newTrainData[(newTrainData.Date == '2021-07-01') & (newTrainData.Asset_ID == 1)]
trainDataFor1JulHigh = trainDataFor1Jul.loc[trainDataFor1Jul[['High']].idxmax()]
trainDataFor1JulLow = trainDataFor1Jul.loc[trainDataFor1Jul[['Low']].idxmin()]

trainDataFor2Jul = newTrainData[(newTrainData.Date == '2021-07-02') & (newTrainData.Asset_ID == 1)]
trainDataFor2JulHigh = trainDataFor2Jul.loc[trainDataFor2Jul[['High']].idxmax()]
trainDataFor2JulLow = trainDataFor2Jul.loc[trainDataFor2Jul[['Low']].idxmin()]

trainDataFor3Jul = newTrainData[(newTrainData.Date == '2021-07-03') & (newTrainData.Asset_ID == 1)]
trainDataFor3JulHigh = trainDataFor3Jul.loc[trainDataFor3Jul[['High']].idxmax()]
trainDataFor3JulLow = trainDataFor3Jul.loc[trainDataFor3Jul[['Low']].idxmin()]

trainDataFor4Jul = newTrainData[(newTrainData.Date == '2021-07-04') & (newTrainData.Asset_ID == 1)]
trainDataFor4JulHigh = trainDataFor4Jul.loc[trainDataFor4Jul[['High']].idxmax()]
trainDataFor4JulLow = trainDataFor4Jul.loc[trainDataFor4Jul[['Low']].idxmin()]

trainDataFor5Jul = newTrainData[(newTrainData.Date == '2021-07-05') & (newTrainData.Asset_ID == 1)]
trainDataFor5JulHigh = trainDataFor5Jul.loc[trainDataFor5Jul[['High']].idxmax()]
trainDataFor5JulLow = trainDataFor5Jul.loc[trainDataFor5Jul[['Low']].idxmin()]

**<p id="22"><li>6 July to 10 July Bitcoin Data</li></p>**

In [ ]:
trainDataFor6Jul = newTrainData[(newTrainData.Date == '2021-07-06') & (newTrainData.Asset_ID == 1)]
trainDataFor6JulHigh = trainDataFor6Jul.loc[trainDataFor6Jul[['High']].idxmax()]
trainDataFor6JulLow = trainDataFor6Jul.loc[trainDataFor6Jul[['Low']].idxmin()]

trainDataFor7Jul = newTrainData[(newTrainData.Date == '2021-07-07') & (newTrainData.Asset_ID == 1)]
trainDataFor7JulHigh = trainDataFor7Jul.loc[trainDataFor7Jul[['High']].idxmax()]
trainDataFor7JulLow = trainDataFor7Jul.loc[trainDataFor7Jul[['Low']].idxmin()]

trainDataFor8Jul = newTrainData[(newTrainData.Date == '2021-07-08') & (newTrainData.Asset_ID == 1)]
trainDataFor8JulHigh = trainDataFor8Jul.loc[trainDataFor8Jul[['High']].idxmax()]
trainDataFor8JulLow = trainDataFor8Jul.loc[trainDataFor8Jul[['Low']].idxmin()]

trainDataFor9Jul = newTrainData[(newTrainData.Date == '2021-07-09') & (newTrainData.Asset_ID == 1)]
trainDataFor9JulHigh = trainDataFor9Jul.loc[trainDataFor9Jul[['High']].idxmax()]
trainDataFor9JulLow = trainDataFor9Jul.loc[trainDataFor9Jul[['Low']].idxmin()]

trainDataFor10Jul = newTrainData[(newTrainData.Date == '2021-07-10') & (newTrainData.Asset_ID == 1)]
trainDataFor10JulHigh = trainDataFor10Jul.loc[trainDataFor10Jul[['High']].idxmax()]
trainDataFor10JulLow = trainDataFor10Jul.loc[trainDataFor10Jul[['Low']].idxmin()]

 **<p id="23"><li>11 July to 15 July Bitcoin Data</li></p>**

In [ ]:
trainDataFor11Jul = newTrainData[(newTrainData.Date == '2021-07-11') & (newTrainData.Asset_ID == 1)]
trainDataFor11JulHigh = trainDataFor11Jul.loc[trainDataFor11Jul[['High']].idxmax()]
trainDataFor11JulLow = trainDataFor11Jul.loc[trainDataFor11Jul[['Low']].idxmin()]

trainDataFor12Jul = newTrainData[(newTrainData.Date == '2021-07-12') & (newTrainData.Asset_ID == 1)]
trainDataFor12JulHigh = trainDataFor12Jul.loc[trainDataFor12Jul[['High']].idxmax()]
trainDataFor12JulLow = trainDataFor12Jul.loc[trainDataFor12Jul[['Low']].idxmin()]

trainDataFor13Jul = newTrainData[(newTrainData.Date == '2021-07-13') & (newTrainData.Asset_ID == 1)]
trainDataFor13JulHigh = trainDataFor13Jul.loc[trainDataFor13Jul[['High']].idxmax()]
trainDataFor13JulLow = trainDataFor13Jul.loc[trainDataFor13Jul[['Low']].idxmin()]

trainDataFor14Jul = newTrainData[(newTrainData.Date == '2021-07-14') & (newTrainData.Asset_ID == 1)]
trainDataFor14JulHigh = trainDataFor14Jul.loc[trainDataFor14Jul[['High']].idxmax()]
trainDataFor14JulLow = trainDataFor14Jul.loc[trainDataFor14Jul[['Low']].idxmin()]

trainDataFor15Jul = newTrainData[(newTrainData.Date == '2021-07-15') & (newTrainData.Asset_ID == 1)]
trainDataFor15JulHigh = trainDataFor15Jul.loc[trainDataFor15Jul[['High']].idxmax()]
trainDataFor15JulLow = trainDataFor15Jul.loc[trainDataFor15Jul[['Low']].idxmin()]

**<p id="24"><li>16 July to 20 July Bitcoin Data</li></p>**

In [ ]:
trainDataFor16Jul = newTrainData[(newTrainData.Date == '2021-07-16') & (newTrainData.Asset_ID == 1)]
trainDataFor16JulHigh = trainDataFor16Jul.loc[trainDataFor16Jul[['High']].idxmax()]
trainDataFor16JulLow = trainDataFor16Jul.loc[trainDataFor16Jul[['Low']].idxmin()]

trainDataFor17Jul = newTrainData[(newTrainData.Date == '2021-07-17') & (newTrainData.Asset_ID == 1)]
trainDataFor17JulHigh = trainDataFor17Jul.loc[trainDataFor17Jul[['High']].idxmax()]
trainDataFor17JulLow = trainDataFor17Jul.loc[trainDataFor17Jul[['Low']].idxmin()]

trainDataFor18Jul = newTrainData[(newTrainData.Date == '2021-07-18') & (newTrainData.Asset_ID == 1)]
trainDataFor18JulHigh = trainDataFor18Jul.loc[trainDataFor18Jul[['High']].idxmax()]
trainDataFor18JulLow = trainDataFor18Jul.loc[trainDataFor18Jul[['Low']].idxmin()]

trainDataFor19Jul = newTrainData[(newTrainData.Date == '2021-07-19') & (newTrainData.Asset_ID == 1)]
trainDataFor19JulHigh = trainDataFor19Jul.loc[trainDataFor19Jul[['High']].idxmax()]
trainDataFor19JulLow = trainDataFor19Jul.loc[trainDataFor19Jul[['Low']].idxmin()]

trainDataFor20Jul = newTrainData[(newTrainData.Date == '2021-07-20') & (newTrainData.Asset_ID == 1)]
trainDataFor20JulHigh = trainDataFor20Jul.loc[trainDataFor20Jul[['High']].idxmax()]
trainDataFor20JulLow = trainDataFor20Jul.loc[trainDataFor20Jul[['Low']].idxmin()]

**<p id="25"><li>21 July to 25 July Bitcoin Data</li></p>**

In [ ]:
trainDataFor21Jul = newTrainData[(newTrainData.Date == '2021-07-21') & (newTrainData.Asset_ID == 1)]
trainDataFor21JulHigh = trainDataFor21Jul.loc[trainDataFor21Jul[['High']].idxmax()]
trainDataFor21JulLow = trainDataFor21Jul.loc[trainDataFor21Jul[['Low']].idxmin()]

trainDataFor22Jul = newTrainData[(newTrainData.Date == '2021-07-22') & (newTrainData.Asset_ID == 1)]
trainDataFor22JulHigh = trainDataFor22Jul.loc[trainDataFor22Jul[['High']].idxmax()]
trainDataFor22JulLow = trainDataFor22Jul.loc[trainDataFor22Jul[['Low']].idxmin()]

trainDataFor23Jul = newTrainData[(newTrainData.Date == '2021-07-23') & (newTrainData.Asset_ID == 1)]
trainDataFor23JulHigh = trainDataFor23Jul.loc[trainDataFor23Jul[['High']].idxmax()]
trainDataFor23JulLow = trainDataFor23Jul.loc[trainDataFor23Jul[['Low']].idxmin()]

trainDataFor24Jul = newTrainData[(newTrainData.Date == '2021-07-24') & (newTrainData.Asset_ID == 1)]
trainDataFor24JulHigh = trainDataFor24Jul.loc[trainDataFor24Jul[['High']].idxmax()]
trainDataFor24JulLow = trainDataFor24Jul.loc[trainDataFor24Jul[['Low']].idxmin()]

trainDataFor25Jul = newTrainData[(newTrainData.Date == '2021-07-25') & (newTrainData.Asset_ID == 1)]
trainDataFor25JulHigh = trainDataFor25Jul.loc[trainDataFor25Jul[['High']].idxmax()]
trainDataFor25JulLow = trainDataFor25Jul.loc[trainDataFor25Jul[['Low']].idxmin()]

 **<p id="26"><li>26 July to 28 July Bitcoin Data</li></p>**

In [ ]:
trainDataFor26Jul = newTrainData[(newTrainData.Date == '2021-07-26') & (newTrainData.Asset_ID == 1)]
trainDataFor26JulHigh = trainDataFor26Jul.loc[trainDataFor26Jul[['High']].idxmax()]
trainDataFor26JulLow = trainDataFor26Jul.loc[trainDataFor26Jul[['Low']].idxmin()]

trainDataFor27Jul = newTrainData[(newTrainData.Date == '2021-07-27') & (newTrainData.Asset_ID == 1)]
trainDataFor27JulHigh = trainDataFor27Jul.loc[trainDataFor27Jul[['High']].idxmax()]
trainDataFor27JulLow = trainDataFor27Jul.loc[trainDataFor27Jul[['Low']].idxmin()]

trainDataFor28Jul = newTrainData[(newTrainData.Date == '2021-07-28') & (newTrainData.Asset_ID == 1)]
trainDataFor28JulHigh = trainDataFor28Jul.loc[trainDataFor28Jul[['High']].idxmax()]
trainDataFor28JulLow = trainDataFor28Jul.loc[trainDataFor28Jul[['Low']].idxmin()]

 **<p id="27"><li>29 July to 31 July Bitcoin Data</li></p>**

In [ ]:
trainDataFor29Jul = newTrainData[(newTrainData.Date == '2021-07-29') & (newTrainData.Asset_ID == 1)]
trainDataFor29JulHigh = trainDataFor29Jul.loc[trainDataFor29Jul[['High']].idxmax()]
trainDataFor29JulLow = trainDataFor29Jul.loc[trainDataFor29Jul[['Low']].idxmin()]

trainDataFor30Jul = newTrainData[(newTrainData.Date == '2021-07-30') & (newTrainData.Asset_ID == 1)]
trainDataFor30JulHigh = trainDataFor30Jul.loc[trainDataFor30Jul[['High']].idxmax()]
trainDataFor30JulLow = trainDataFor30Jul.loc[trainDataFor30Jul[['Low']].idxmin()]

trainDataFor31Jul = newTrainData[(newTrainData.Date == '2021-07-31') & (newTrainData.Asset_ID == 1)]
trainDataFor31JulHigh = trainDataFor31Jul.loc[trainDataFor31Jul[['High']].idxmax()]
trainDataFor31JulLow = trainDataFor31Jul.loc[trainDataFor31Jul[['Low']].idxmin()]

**<p id="28"><li>21 August to 5 August Bitcoin Data</li></p>**

In [ ]:
trainDataFor1Aug = newTrainData[(newTrainData.Date == '2021-08-01') & (newTrainData.Asset_ID == 1)]
trainDataFor1AugHigh = trainDataFor1Aug.loc[trainDataFor1Aug[['High']].idxmax()]
trainDataFor1AugLow = trainDataFor1Aug.loc[trainDataFor1Aug[['Low']].idxmin()]

trainDataFor2Aug = newTrainData[(newTrainData.Date == '2021-08-02') & (newTrainData.Asset_ID == 1)]
trainDataFor2AugHigh = trainDataFor2Aug.loc[trainDataFor2Aug[['High']].idxmax()]
trainDataFor2AugLow = trainDataFor2Aug.loc[trainDataFor2Aug[['Low']].idxmin()]

trainDataFor3Aug = newTrainData[(newTrainData.Date == '2021-08-03') & (newTrainData.Asset_ID == 1)]
trainDataFor3AugHigh = trainDataFor3Aug.loc[trainDataFor3Aug[['High']].idxmax()]
trainDataFor3AugLow = trainDataFor3Aug.loc[trainDataFor3Aug[['Low']].idxmin()]

trainDataFor4Aug = newTrainData[(newTrainData.Date == '2021-08-04') & (newTrainData.Asset_ID == 1)]
trainDataFor4AugHigh = trainDataFor4Aug.loc[trainDataFor4Aug[['High']].idxmax()]
trainDataFor4AugLow = trainDataFor4Aug.loc[trainDataFor4Aug[['Low']].idxmin()]

trainDataFor5Aug = newTrainData[(newTrainData.Date == '2021-08-05') & (newTrainData.Asset_ID == 1)]
trainDataFor5AugHigh = trainDataFor5Aug.loc[trainDataFor5Aug[['High']].idxmax()]
trainDataFor5AugLow = trainDataFor5Aug.loc[trainDataFor5Aug[['Low']].idxmin()]

**<p id="29"><li>6 August to 10 August Bitcoin Data</li></p>**

In [ ]:
trainDataFor6Aug = newTrainData[(newTrainData.Date == '2021-08-06') & (newTrainData.Asset_ID == 1)]
trainDataFor6AugHigh = trainDataFor6Aug.loc[trainDataFor6Aug[['High']].idxmax()]
trainDataFor6AugLow = trainDataFor6Aug.loc[trainDataFor6Aug[['Low']].idxmin()]

trainDataFor7Aug = newTrainData[(newTrainData.Date == '2021-08-07') & (newTrainData.Asset_ID == 1)]
trainDataFor7AugHigh = trainDataFor7Aug.loc[trainDataFor7Aug[['High']].idxmax()]
trainDataFor7AugLow = trainDataFor7Aug.loc[trainDataFor7Aug[['Low']].idxmin()]

trainDataFor8Aug = newTrainData[(newTrainData.Date == '2021-08-08') & (newTrainData.Asset_ID == 1)]
trainDataFor8AugHigh = trainDataFor8Aug.loc[trainDataFor8Aug[['High']].idxmax()]
trainDataFor8AugLow = trainDataFor8Aug.loc[trainDataFor8Aug[['Low']].idxmin()]

trainDataFor9Aug = newTrainData[(newTrainData.Date == '2021-08-09') & (newTrainData.Asset_ID == 1)]
trainDataFor9AugHigh = trainDataFor9Aug.loc[trainDataFor9Aug[['High']].idxmax()]
trainDataFor9AugLow = trainDataFor9Aug.loc[trainDataFor9Aug[['Low']].idxmin()]

trainDataFor10Aug = newTrainData[(newTrainData.Date == '2021-08-10') & (newTrainData.Asset_ID == 1)]
trainDataFor10AugHigh = trainDataFor10Aug.loc[trainDataFor10Aug[['High']].idxmax()]
trainDataFor10AugLow = trainDataFor10Aug.loc[trainDataFor10Aug[['Low']].idxmin()]

**<p id="30"><li>11 August to 15 August Bitcoin Data</li></p>**

In [ ]:
trainDataFor11Aug = newTrainData[(newTrainData.Date == '2021-08-11') & (newTrainData.Asset_ID == 1)]
trainDataFor11AugHigh = trainDataFor11Aug.loc[trainDataFor11Aug[['High']].idxmax()]
trainDataFor11AugLow = trainDataFor11Aug.loc[trainDataFor11Aug[['Low']].idxmin()]

trainDataFor12Aug = newTrainData[(newTrainData.Date == '2021-08-12') & (newTrainData.Asset_ID == 1)]
trainDataFor12AugHigh = trainDataFor12Aug.loc[trainDataFor12Aug[['High']].idxmax()]
trainDataFor12AugLow = trainDataFor12Aug.loc[trainDataFor12Aug[['Low']].idxmin()]

trainDataFor13Aug = newTrainData[(newTrainData.Date == '2021-08-13') & (newTrainData.Asset_ID == 1)]
trainDataFor13AugHigh = trainDataFor13Aug.loc[trainDataFor13Aug[['High']].idxmax()]
trainDataFor13AugLow = trainDataFor13Aug.loc[trainDataFor13Aug[['Low']].idxmin()]

trainDataFor14Aug = newTrainData[(newTrainData.Date == '2021-08-14') & (newTrainData.Asset_ID == 1)]
trainDataFor14AugHigh = trainDataFor14Aug.loc[trainDataFor14Aug[['High']].idxmax()]
trainDataFor14AugLow = trainDataFor14Aug.loc[trainDataFor14Aug[['Low']].idxmin()]

trainDataFor15Aug = newTrainData[(newTrainData.Date == '2021-08-15') & (newTrainData.Asset_ID == 1)]
trainDataFor15AugHigh = trainDataFor15Aug.loc[trainDataFor15Aug[['High']].idxmax()]
trainDataFor15AugLow = trainDataFor15Aug.loc[trainDataFor15Aug[['Low']].idxmin()]

**<p id="31"><li>16 August to 20 August Bitcoin Data</li></p>**

In [ ]:
trainDataFor16Aug = newTrainData[(newTrainData.Date == '2021-08-16') & (newTrainData.Asset_ID == 1)]
trainDataFor16AugHigh = trainDataFor16Aug.loc[trainDataFor16Aug[['High']].idxmax()]
trainDataFor16AugLow = trainDataFor16Aug.loc[trainDataFor16Aug[['Low']].idxmin()]

trainDataFor17Aug = newTrainData[(newTrainData.Date == '2021-08-17') & (newTrainData.Asset_ID == 1)]
trainDataFor17AugHigh = trainDataFor17Aug.loc[trainDataFor17Aug[['High']].idxmax()]
trainDataFor17AugLow = trainDataFor17Aug.loc[trainDataFor17Aug[['Low']].idxmin()]

trainDataFor18Aug = newTrainData[(newTrainData.Date == '2021-08-18') & (newTrainData.Asset_ID == 1)]
trainDataFor18AugHigh = trainDataFor18Aug.loc[trainDataFor18Aug[['High']].idxmax()]
trainDataFor18AugLow = trainDataFor18Aug.loc[trainDataFor18Aug[['Low']].idxmin()]

trainDataFor19Aug = newTrainData[(newTrainData.Date == '2021-08-19') & (newTrainData.Asset_ID == 1)]
trainDataFor19AugHigh = trainDataFor19Aug.loc[trainDataFor19Aug[['High']].idxmax()]
trainDataFor19AugLow = trainDataFor19Aug.loc[trainDataFor19Aug[['Low']].idxmin()]

trainDataFor20Aug = newTrainData[(newTrainData.Date == '2021-08-20') & (newTrainData.Asset_ID == 1)]
trainDataFor20AugHigh = trainDataFor20Aug.loc[trainDataFor20Aug[['High']].idxmax()]
trainDataFor20AugLow = trainDataFor20Aug.loc[trainDataFor20Aug[['Low']].idxmin()]

**<p id="32"><li>21 August to 25 August Bitcoin Data</li></p>**

In [ ]:
trainDataFor21Aug = newTrainData[(newTrainData.Date == '2021-08-21') & (newTrainData.Asset_ID == 1)]
trainDataFor21AugHigh = trainDataFor21Aug.loc[trainDataFor21Aug[['High']].idxmax()]
trainDataFor21AugLow = trainDataFor21Aug.loc[trainDataFor21Aug[['Low']].idxmin()]

trainDataFor22Aug = newTrainData[(newTrainData.Date == '2021-08-22') & (newTrainData.Asset_ID == 1)]
trainDataFor22AugHigh = trainDataFor22Aug.loc[trainDataFor22Aug[['High']].idxmax()]
trainDataFor22AugLow = trainDataFor22Aug.loc[trainDataFor22Aug[['Low']].idxmin()]

trainDataFor23Aug = newTrainData[(newTrainData.Date == '2021-08-23') & (newTrainData.Asset_ID == 1)]
trainDataFor23AugHigh = trainDataFor23Aug.loc[trainDataFor23Aug[['High']].idxmax()]
trainDataFor23AugLow = trainDataFor23Aug.loc[trainDataFor23Aug[['Low']].idxmin()]

trainDataFor24Aug = newTrainData[(newTrainData.Date == '2021-08-24') & (newTrainData.Asset_ID == 1)]
trainDataFor24AugHigh = trainDataFor24Aug.loc[trainDataFor24Aug[['High']].idxmax()]
trainDataFor24AugLow = trainDataFor24Aug.loc[trainDataFor24Aug[['Low']].idxmin()]

trainDataFor25Aug = newTrainData[(newTrainData.Date == '2021-08-25') & (newTrainData.Asset_ID == 1)]
trainDataFor25AugHigh = trainDataFor25Aug.loc[trainDataFor25Aug[['High']].idxmax()]
trainDataFor25AugLow = trainDataFor25Aug.loc[trainDataFor25Aug[['Low']].idxmin()]

**<p id="33"><li>26 August to 28 August Bitcoin Data</li></p>**

In [ ]:
trainDataFor26Aug = newTrainData[(newTrainData.Date == '2021-08-26') & (newTrainData.Asset_ID == 1)]
trainDataFor26AugHigh = trainDataFor26Aug.loc[trainDataFor26Aug[['High']].idxmax()]
trainDataFor26AugLow = trainDataFor26Aug.loc[trainDataFor26Aug[['Low']].idxmin()]

trainDataFor27Aug = newTrainData[(newTrainData.Date == '2021-08-27') & (newTrainData.Asset_ID == 1)]
trainDataFor27AugHigh = trainDataFor27Aug.loc[trainDataFor27Aug[['High']].idxmax()]
trainDataFor27AugLow = trainDataFor27Aug.loc[trainDataFor27Aug[['Low']].idxmin()]

trainDataFor28Aug = newTrainData[(newTrainData.Date == '2021-08-28') & (newTrainData.Asset_ID == 1)]
trainDataFor28AugHigh = trainDataFor28Aug.loc[trainDataFor28Aug[['High']].idxmax()]
trainDataFor28AugLow = trainDataFor28Aug.loc[trainDataFor28Aug[['Low']].idxmin()]

**<p id="34"><li>29 August to 31 August Bitcoin Data</li></p>**

In [ ]:
trainDataFor29Aug = newTrainData[(newTrainData.Date == '2021-08-29') & (newTrainData.Asset_ID == 1)]
trainDataFor29AugHigh = trainDataFor29Aug.loc[trainDataFor29Aug[['High']].idxmax()]
trainDataFor29AugLow = trainDataFor29Aug.loc[trainDataFor29Aug[['Low']].idxmin()]

trainDataFor30Aug = newTrainData[(newTrainData.Date == '2021-08-30') & (newTrainData.Asset_ID == 1)]
trainDataFor30AugHigh = trainDataFor30Aug.loc[trainDataFor30Aug[['High']].idxmax()]
trainDataFor30AugLow = trainDataFor30Aug.loc[trainDataFor30Aug[['Low']].idxmin()]

trainDataFor31Aug = newTrainData[(newTrainData.Date == '2021-08-31') & (newTrainData.Asset_ID == 1)]
trainDataFor31AugHigh = trainDataFor31Aug.loc[trainDataFor31Aug[['High']].idxmax()]
trainDataFor31AugLow = trainDataFor31Aug.loc[trainDataFor31Aug[['Low']].idxmin()]

**<p id="35"><li>1 September to 5 September Bitcoin Data</li></p>**

In [ ]:
trainDataFor1Sep = newTrainData[(newTrainData.Date == '2021-09-01') & (newTrainData.Asset_ID == 1)]
trainDataFor1SepHigh = trainDataFor1Sep.loc[trainDataFor1Sep[['High']].idxmax()]
trainDataFor1SepLow = trainDataFor1Sep.loc[trainDataFor1Sep[['Low']].idxmin()]

trainDataFor2Sep = newTrainData[(newTrainData.Date == '2021-09-02') & (newTrainData.Asset_ID == 1)]
trainDataFor2SepHigh = trainDataFor2Sep.loc[trainDataFor2Sep[['High']].idxmax()]
trainDataFor2SepLow = trainDataFor2Sep.loc[trainDataFor2Sep[['Low']].idxmin()]

trainDataFor3Sep = newTrainData[(newTrainData.Date == '2021-09-03') & (newTrainData.Asset_ID == 1)]
trainDataFor3SepHigh = trainDataFor3Sep.loc[trainDataFor3Sep[['High']].idxmax()]
trainDataFor3SepLow = trainDataFor3Sep.loc[trainDataFor3Sep[['Low']].idxmin()]

trainDataFor4Sep = newTrainData[(newTrainData.Date == '2021-09-04') & (newTrainData.Asset_ID == 1)]
trainDataFor4SepHigh = trainDataFor4Sep.loc[trainDataFor4Sep[['High']].idxmax()]
trainDataFor4SepLow = trainDataFor4Sep.loc[trainDataFor4Sep[['Low']].idxmin()]

trainDataFor5Sep = newTrainData[(newTrainData.Date == '2021-09-05') & (newTrainData.Asset_ID == 1)]
trainDataFor5SepHigh = trainDataFor5Sep.loc[trainDataFor5Sep[['High']].idxmax()]
trainDataFor5SepLow = trainDataFor5Sep.loc[trainDataFor5Sep[['Low']].idxmin()]

**<p id="36"><li>6 September to 10 September Bitcoin Data</li></p>**

In [ ]:
trainDataFor6Sep = newTrainData[(newTrainData.Date == '2021-09-06') & (newTrainData.Asset_ID == 1)]
trainDataFor6SepHigh = trainDataFor6Sep.loc[trainDataFor6Sep[['High']].idxmax()]
trainDataFor6SepLow = trainDataFor6Sep.loc[trainDataFor6Sep[['Low']].idxmin()]

trainDataFor7Sep = newTrainData[(newTrainData.Date == '2021-09-07') & (newTrainData.Asset_ID == 1)]
trainDataFor7SepHigh = trainDataFor7Sep.loc[trainDataFor7Sep[['High']].idxmax()]
trainDataFor7SepLow = trainDataFor7Sep.loc[trainDataFor7Sep[['Low']].idxmin()]

trainDataFor8Sep = newTrainData[(newTrainData.Date == '2021-09-08') & (newTrainData.Asset_ID == 1)]
trainDataFor8SepHigh = trainDataFor8Sep.loc[trainDataFor8Sep[['High']].idxmax()]
trainDataFor8SepLow = trainDataFor8Sep.loc[trainDataFor8Sep[['Low']].idxmin()]

trainDataFor9Sep = newTrainData[(newTrainData.Date == '2021-09-09') & (newTrainData.Asset_ID == 1)]
trainDataFor9SepHigh = trainDataFor9Sep.loc[trainDataFor9Sep[['High']].idxmax()]
trainDataFor9SepLow = trainDataFor9Sep.loc[trainDataFor9Sep[['Low']].idxmin()]

trainDataFor10Sep = newTrainData[(newTrainData.Date == '2021-09-10') & (newTrainData.Asset_ID == 1)]
trainDataFor10SepHigh = trainDataFor10Sep.loc[trainDataFor10Sep[['High']].idxmax()]
trainDataFor10SepLow = trainDataFor10Sep.loc[trainDataFor10Sep[['Low']].idxmin()]

**<p id="37"><li>11 September to 15 September Bitcoin Data</li></p>**

In [ ]:
trainDataFor11Sep = newTrainData[(newTrainData.Date == '2021-09-11') & (newTrainData.Asset_ID == 1)]
trainDataFor11SepHigh = trainDataFor11Sep.loc[trainDataFor11Sep[['High']].idxmax()]
trainDataFor11SepLow = trainDataFor11Sep.loc[trainDataFor11Sep[['Low']].idxmin()]

trainDataFor12Sep = newTrainData[(newTrainData.Date == '2021-09-12') & (newTrainData.Asset_ID == 1)]
trainDataFor12SepHigh = trainDataFor12Sep.loc[trainDataFor12Sep[['High']].idxmax()]
trainDataFor12SepLow = trainDataFor12Sep.loc[trainDataFor12Sep[['Low']].idxmin()]

trainDataFor13Sep = newTrainData[(newTrainData.Date == '2021-09-13') & (newTrainData.Asset_ID == 1)]
trainDataFor13SepHigh = trainDataFor13Sep.loc[trainDataFor13Sep[['High']].idxmax()]
trainDataFor13SepLow = trainDataFor13Sep.loc[trainDataFor13Sep[['Low']].idxmin()]

trainDataFor14Sep = newTrainData[(newTrainData.Date == '2021-09-14') & (newTrainData.Asset_ID == 1)]
trainDataFor14SepHigh = trainDataFor14Sep.loc[trainDataFor14Sep[['High']].idxmax()]
trainDataFor14SepLow = trainDataFor14Sep.loc[trainDataFor14Sep[['Low']].idxmin()]

trainDataFor15Sep = newTrainData[(newTrainData.Date == '2021-09-15') & (newTrainData.Asset_ID == 1)]
trainDataFor15SepHigh = trainDataFor15Sep.loc[trainDataFor15Sep[['High']].idxmax()]
trainDataFor15SepLow = trainDataFor15Sep.loc[trainDataFor15Sep[['Low']].idxmin()]

**<p id="38"><li>16 September to 18 September Bitcoin Data</li></p>**

In [ ]:
trainDataFor16Sep = newTrainData[(newTrainData.Date == '2021-09-16') & (newTrainData.Asset_ID == 1)]
trainDataFor16SepHigh = trainDataFor16Sep.loc[trainDataFor16Sep[['High']].idxmax()]
trainDataFor16SepLow = trainDataFor16Sep.loc[trainDataFor16Sep[['Low']].idxmin()]

trainDataFor17Sep = newTrainData[(newTrainData.Date == '2021-09-17') & (newTrainData.Asset_ID == 1)]
trainDataFor17SepHigh = trainDataFor17Sep.loc[trainDataFor17Sep[['High']].idxmax()]
trainDataFor17SepLow = trainDataFor17Sep.loc[trainDataFor17Sep[['Low']].idxmin()]

trainDataFor18Sep = newTrainData[(newTrainData.Date == '2021-09-18') & (newTrainData.Asset_ID == 1)]
trainDataFor18SepHigh = trainDataFor18Sep.loc[trainDataFor18Sep[['High']].idxmax()]
trainDataFor18SepLow = trainDataFor18Sep.loc[trainDataFor18Sep[['Low']].idxmin()]

**<p id="39"><li>19 September to 21 September Bitcoin Data</li></p>**

In [ ]:
trainDataFor19Sep = newTrainData[(newTrainData.Date == '2021-09-19') & (newTrainData.Asset_ID == 1)]
trainDataFor19SepHigh = trainDataFor19Sep.loc[trainDataFor19Sep[['High']].idxmax()]
trainDataFor19SepLow = trainDataFor19Sep.loc[trainDataFor19Sep[['Low']].idxmin()]

trainDataFor20Sep = newTrainData[(newTrainData.Date == '2021-09-20') & (newTrainData.Asset_ID == 1)]
trainDataFor20SepHigh = trainDataFor20Sep.loc[trainDataFor20Sep[['High']].idxmax()]
trainDataFor20SepLow = trainDataFor20Sep.loc[trainDataFor20Sep[['Low']].idxmin()]

trainDataFor21Sep = newTrainData[(newTrainData.Date == '2021-09-21') & (newTrainData.Asset_ID == 1)]
trainDataFor21SepHigh = trainDataFor21Sep.loc[trainDataFor21Sep[['High']].idxmax()]
trainDataFor21SepLow = trainDataFor21Sep.loc[trainDataFor21Sep[['Low']].idxmin()]

<a id="40"></a> <br>
# Data for ARIMA Model

In [ ]:
frames = [
    trainDataFor24JunHigh, trainDataFor24JunLow,
    trainDataFor25JunHigh, trainDataFor25JunLow,
    trainDataFor26JunHigh, trainDataFor26JunLow,
    trainDataFor27JunHigh, trainDataFor27JunLow,
    trainDataFor28JunHigh, trainDataFor28JunLow,
    trainDataFor29JunHigh, trainDataFor29JunLow,
    trainDataFor30JunHigh, trainDataFor30JunLow,
    trainDataFor1JulHigh, trainDataFor1JulLow,
    trainDataFor2JulHigh, trainDataFor2JulLow,
    trainDataFor3JulHigh, trainDataFor3JulLow,
    trainDataFor4JulHigh, trainDataFor4JulLow,
    trainDataFor5JulHigh, trainDataFor5JulLow,
    trainDataFor6JulHigh, trainDataFor6JulLow,
    trainDataFor7JulHigh, trainDataFor7JulLow,
    trainDataFor8JulHigh, trainDataFor8JulLow,
    trainDataFor9JulHigh, trainDataFor9JulLow,
    trainDataFor10JulHigh, trainDataFor10JulLow,
    trainDataFor11JulHigh, trainDataFor11JulLow,
    trainDataFor12JulHigh, trainDataFor12JulLow,
    trainDataFor13JulHigh, trainDataFor13JulLow,
    trainDataFor14JulHigh, trainDataFor14JulLow,
    trainDataFor15JulHigh, trainDataFor15JulLow,
    trainDataFor16JulHigh, trainDataFor16JulLow,
    trainDataFor17JulHigh, trainDataFor17JulLow,
    trainDataFor18JulHigh, trainDataFor18JulLow,
    trainDataFor19JulHigh, trainDataFor19JulLow,
    trainDataFor20JulHigh, trainDataFor20JulLow,
    trainDataFor21JulHigh, trainDataFor21JulLow,
    trainDataFor22JulHigh, trainDataFor22JulLow,
    trainDataFor23JulHigh, trainDataFor23JulLow,
    trainDataFor24JulHigh, trainDataFor24JulLow,
    trainDataFor25JulHigh, trainDataFor25JulLow,
    trainDataFor26JulHigh, trainDataFor26JulLow,
    trainDataFor27JulHigh, trainDataFor27JulLow,
    trainDataFor28JulHigh, trainDataFor28JulLow,
    trainDataFor29JulHigh, trainDataFor29JulLow,
    trainDataFor30JulHigh, trainDataFor30JulLow,
    trainDataFor31JulHigh, trainDataFor31JulLow,
    trainDataFor1AugHigh, trainDataFor1AugLow,
    trainDataFor2AugHigh, trainDataFor2AugLow,
    trainDataFor3AugHigh, trainDataFor3AugLow,
    trainDataFor4AugHigh, trainDataFor4AugLow,
    trainDataFor5AugHigh, trainDataFor5AugLow,
    trainDataFor6AugHigh, trainDataFor6AugLow,
    trainDataFor7AugHigh, trainDataFor7AugLow,
    trainDataFor8AugHigh, trainDataFor8AugLow,
    trainDataFor9AugHigh, trainDataFor9AugLow,
    trainDataFor10AugHigh, trainDataFor10AugLow,
    trainDataFor11AugHigh, trainDataFor11AugLow,
    trainDataFor12AugHigh, trainDataFor12AugLow,
    trainDataFor13AugHigh, trainDataFor13AugLow,
    trainDataFor14AugHigh, trainDataFor14AugLow,
    trainDataFor15AugHigh, trainDataFor15AugLow,
    trainDataFor16AugHigh, trainDataFor16AugLow,
    trainDataFor17AugHigh, trainDataFor17AugLow,
    trainDataFor18AugHigh, trainDataFor18AugLow,
    trainDataFor19AugHigh, trainDataFor19AugLow,
    trainDataFor20AugHigh, trainDataFor20AugLow,
    trainDataFor21AugHigh, trainDataFor21AugLow,
    trainDataFor22AugHigh, trainDataFor22AugLow,
    trainDataFor23AugHigh, trainDataFor23AugLow,
    trainDataFor24AugHigh, trainDataFor24AugLow,
    trainDataFor25AugHigh, trainDataFor25AugLow,
    trainDataFor26AugHigh, trainDataFor26AugLow,
    trainDataFor27AugHigh, trainDataFor27AugLow,
    trainDataFor28AugHigh, trainDataFor28AugLow,
    trainDataFor29AugHigh, trainDataFor29AugLow,
    trainDataFor30AugHigh, trainDataFor30AugLow,
    trainDataFor31AugHigh, trainDataFor31AugLow,
    trainDataFor1SepHigh, trainDataFor1SepLow,
    trainDataFor2SepHigh, trainDataFor2SepLow,
    trainDataFor3SepHigh, trainDataFor3SepLow,
    trainDataFor4SepHigh, trainDataFor4SepLow,
    trainDataFor5SepHigh, trainDataFor5SepLow,
    trainDataFor6SepHigh, trainDataFor6SepLow,
    trainDataFor7SepHigh, trainDataFor7SepLow,
    trainDataFor8SepHigh, trainDataFor8SepLow,
    trainDataFor9SepHigh, trainDataFor9SepLow,
    trainDataFor10SepHigh, trainDataFor10SepLow,
    trainDataFor11SepHigh, trainDataFor11SepLow,
    trainDataFor12SepHigh, trainDataFor12SepLow,
    trainDataFor13SepHigh, trainDataFor13SepLow,
    trainDataFor14SepHigh, trainDataFor14SepLow,
    trainDataFor15SepHigh, trainDataFor15SepLow,
    trainDataFor16SepHigh, trainDataFor16SepLow,
    trainDataFor17SepHigh, trainDataFor17SepLow,
    trainDataFor18SepHigh, trainDataFor18SepLow,
    trainDataFor19SepHigh, trainDataFor19SepLow,
    trainDataFor20SepHigh, trainDataFor20SepLow,
    trainDataFor21SepHigh, trainDataFor21SepLow,
]

result = pd.concat(frames)
result.set_index(['Date'], inplace=True)

 <a id="41"></a> <br>
# Identifying P Value with ADF

In [ ]:
resultForP = adfuller(result.Close.dropna())
print('ADF Statistic: %f' % resultForP[0])
print('p-value: %f' % resultForP[1])

<a id="42"></a> <br>
# 1st Order Differencing & 2nd Order Differencing Autocorrelation

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (20,5))
ax1.plot(result.Close,color="#ff8c00")
plt.grid(b=True,axis = 'y')
ax1.set_title('Original Series')
ax1.grid(b=True,axis = 'y')
sm.graphics.tsa.plot_acf(result.Close, ax=ax2,color='black',vlines_kwargs={"colors": "#ff8c00"})

diff1 = result.Close.diff().dropna()
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(20,5))
ax1.plot(diff1,color="#004C99")
plt.grid(b=True,axis = 'y')
ax1.set_title("1st Order Differencing")
ax1.grid(b=True,axis = 'x')
sm.graphics.tsa.plot_acf(diff1, ax=ax2,color="black",vlines_kwargs={"colors": "#004C99"})

diff2 = result.Close.diff().diff().dropna()
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(20,5))
ax1.plot(diff2,"g-")
plt.grid(b=True,axis = 'y')
ax1.set_title("2nd Order Differencing")
ax1.grid(b=True,axis = 'x')
my_color="green"
sm.graphics.tsa.plot_acf(diff2, ax=ax2,color="black",vlines_kwargs={"colors": "green"})

plt.show()

<a id="43"></a> <br>
# 1st Order Partial Differencing Partial Autocorrelation

In [ ]:
diffp1 = result.Close.diff().dropna()
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(20,5))
ax1.plot(diffp1,color="#004C99")
plt.grid(b=True,axis = 'y')
ax1.set_title("1st Order Partial Differencing")
ax1.grid(b=True,axis = 'x')
ax2.set_ylim(0,1)
sm.graphics.tsa.plot_pacf(diffp1, ax=ax2,color="black",vlines_kwargs={"colors": "#004C99"})
plt.show()

<a id="44"></a> <br>
# 1st Order Partial Differencing Autocorrelation

In [ ]:
diffp1 = result.Close.diff().dropna()
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(20,5))
ax1.plot(diffp1,color="#004C99")
plt.grid(b=True,axis = 'y')
ax1.set_title("1st Order Partial Differencing")
ax1.grid(b=True,axis = 'x')
ax2.set_ylim(0,1)
sm.graphics.tsa.plot_acf(diffp1, ax=ax2,color="black",vlines_kwargs={"colors": "#004C99"})
plt.show()

<a id="45"></a> <br>
# ARIMA Model

In [ ]:
model = sm.tsa.arima.ARIMA(result.Close, order=(2,1,1))
resultForARIMA = model.fit(method_kwargs={"warn_convergence": False})
print(resultForARIMA.summary())

<a id="46"></a> <br>
# Residuals Plotting for ARIMA

In [ ]:
residuals = pd.DataFrame(resultForARIMA.resid)
fig, ax = plt.subplots(1,2,figsize=(20,5))
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

<a id="47"></a> <br>
# Data of RMSE

In [ ]:
X = result.Close
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
for t in range(len(test)):
    model = ARIMA(history, order=(2,1,0))
    model_fit = model.fit(method_kwargs={"warn_convergence": False})
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)

<a id="48"></a> <br>
# Forecasting Bitcoin Data

In [ ]:
plt.figure(figsize=(12,5), dpi=100)
plt.xticks(rotation = 90)
plt.plot(predictions, label='forecast',color='green')
plt.plot(test, label='actual',color = 'red')
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()